## shell

Shell命令是操作系统中用于在命令行界面（CLI）中执行各种操作的工具。它们可以用于文件和目录管理（如创建、删除、移动和复制）、查看和编辑文本文件、管理系统权限和所有权、监控系统资源、进行网络通信和远程管理，以及安装和更新软件包等。掌握Shell命令可以极大地提高工作效率和系统的管理能力。

pip install langchain-experimental

In [5]:
from langchain.tools import ShellTool

shell_tool = ShellTool()

shell_tool.run(
    {
        "commands": ["open -a 'Google Chrome' 'http://www.baidu.com/s?wd=仙剑四'"]
    }
)

Executing command:
 ["open -a 'Google Chrome' 'http://www.baidu.com/s?wd=仙剑四'"]


/Users/libing/miniconda3/envs/kk_langchain/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


''

In [7]:
from langchain.agents import tool
from typing import List

@tool
def run_shell(commands: str) -> int:
    """在电脑上运行shell的命令操控电脑，例如输入command命令打开浏览器用百度搜索郭德纲，
    则command命令为 open -a 'Google Chrome' 'http://www.baidu.com/s?wd=郭德纲'
    """
    return shell_tool.run({"commands": [commands]})

run_shell.run('open -a "Google Chrome" "http://www.baidu.com/s?wd=郭德纲"')

Executing command:
 ['open -a "Google Chrome" "http://www.baidu.com/s?wd=郭德纲"']


/Users/libing/miniconda3/envs/kk_langchain/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


''

In [8]:
run_shell.run('mkdir kk_test')

Executing command:
 ['mkdir kk_test']


/Users/libing/miniconda3/envs/kk_langchain/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


''

## 加载LLM

首先，让我们加载将用于控制代理的语言模型。

In [10]:
from langchain_openai import ChatOpenAI
import os

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=4096)

## 定义工具

接下来，让我们定义一些要使用的工具。让我们编写一个非常简单的 Python 函数来计算传入的单词的长度。

请注意，这里我们使用的函数文档字符串非常重要。

In [11]:
from langchain.agents import tool

@tool
def word_length(word: str) -> int:
    """计算传入的单词的长度"""
    return len(word)

word_length.invoke("hello")

5

In [12]:
import requests

@tool
def get_weather(location: str) -> dict:
    """根据城市获取天气数据"""
    api_key = "SKcA5FGgmLvN7faJi"
    city_codes = {
        "西安": "xian",
        "北京": "beijing",
        "上海": "shanghai",
        "广州": "guangzhou",
        "深圳": "shenzhen",
        "杭州": "hangzhou",
        "成都": "chengdu",
        "武汉": "wuhan"
    }
    
    # 处理城市名称
    location = location.replace("市", "").replace("区", "")
    location = city_codes.get(location, location)  # 如果有映射就用映射，没有就用原名
    
    
    url = f"https://api.seniverse.com/v3/weather/now.json?key={api_key}&location={location}&language=zh-Hans&unit=c"
    try:
        response = requests.get(url)
        #print(location)
        if response.status_code == 200:
            data = response.json()
            #print(data)
            weather = {
                'description':data['results'][0]["now"]["text"],
                'temperature':data['results'][0]["now"]["temperature"]
            }
            return weather
        else:
            raise Exception(f"失败接收天气信息：{response.status_code}")
    except Exception as e:
        return {"description": "获取天气信息失败", "detail": str(e)}
            
get_weather.invoke("西安")

{'description': '阴', 'temperature': '4'}

In [13]:
tools = [word_length, get_weather, run_shell]
tools

[StructuredTool(name='word_length', description='计算传入的单词的长度', args_schema=<class 'langchain_core.utils.pydantic.word_length'>, func=<function word_length at 0x13c608670>),
 StructuredTool(name='get_weather', description='根据城市获取天气数据', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x13c6097e0>),
 StructuredTool(name='run_shell', description="在电脑上运行shell的命令操控电脑，例如输入command命令打开浏览器用百度搜索郭德纲，\n    则command命令为 open -a 'Google Chrome' 'http://www.baidu.com/s?wd=郭德纲'", args_schema=<class 'langchain_core.utils.pydantic.run_shell'>, func=<function run_shell at 0x12cc637f0>)]

## 创建提示

现在让我们创建提示。由于 OpenAI 函数调用针对工具使用进行了微调，因此我们几乎不需要任何关于如何推理或如何输出格式的说明。我们将只有两个输入变量： input 和 agent_scratchpad 。 input 应为包含用户目标的字符串。 agent_scratchpad 应该是包含先前代理工具调用和相应工具输出的消息序列。

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

promptTemplate = """尽可能的帮助用户回答任何问题。
您可以使用以下工具来帮忙解决问题，如果已经知道答案，也可以直接回答：
{tools}

回复格式说明：
----------------------------------

回复我时，请以以下两种格式之一输出问题：

选项1：如果您希望人类使用工具，请使用此选项。
采用以下JSON模式格式化的回复内容：

```json
{{
    "reason": string, \\ 叙述使用工具的原因
    "action": string, \\ 要使用的工具。 必须是 {tool_names} 之一
    "action_input": string \\ 工具的输入
}}
```

选项2：如果您认为你自己已经有了答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。
采用以下JSON模式格式化的回复内容：

```json
{{
    "action": "Final Answer",
    "answer": string \\ 最终回复问题的答案放在这里！
}}
```

用户输入
----------------------------------
这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：

{input}
"""
promptTemplate

'尽可能的帮助用户回答任何问题。\n您可以使用以下工具来帮忙解决问题，如果已经知道答案，也可以直接回答：\n{tools}\n\n回复格式说明：\n----------------------------------\n\n回复我时，请以以下两种格式之一输出问题：\n\n选项1：如果您希望人类使用工具，请使用此选项。\n采用以下JSON模式格式化的回复内容：\n\n```json\n{{\n    "reason": string, \\ 叙述使用工具的原因\n    "action": string, \\ 要使用的工具。 必须是 {tool_names} 之一\n    "action_input": string \\ 工具的输入\n}}\n```\n\n选项2：如果您认为你自己已经有了答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。\n采用以下JSON模式格式化的回复内容：\n\n```json\n{{\n    "action": "Final Answer",\n    "answer": string \\ 最终回复问题的答案放在这里！\n}}\n```\n\n用户输入\n----------------------------------\n这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：\n\n{input}\n'

In [16]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是非常强大的助手，你可以使用各种工具来完成人类交给的问题和任务。"),
    ("user", promptTemplate),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMess

In [17]:
from langchain.tools.render import render_text_description

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools])
)
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Ann

In [18]:
render_text_description(tools)

"word_length(word: str) -> int - 计算传入的单词的长度\nget_weather(location: str) -> dict - 根据城市获取天气数据\nrun_shell(commands: str) -> int - 在电脑上运行shell的命令操控电脑，例如输入command命令打开浏览器用百度搜索郭德纲，\n    则command命令为 open -a 'Google Chrome' 'http://www.baidu.com/s?wd=郭德纲'"

## 将工具绑定到LLM

在这种情况下，我们依赖于 OpenAI 工具调用 LLMs，它将工具作为一个单独的参数，并经过专门训练，知道何时调用这些工具。

要将我们的工具传递给代理，我们只需要将它们格式化为 OpenAI 工具格式并将它们传递给我们的模型。（通过 bind 对函数进行 -ing，我们确保每次调用模型时都传入它们。

## 创建代理

将这些部分放在一起，我们现在可以创建代理。我们将导入最后两个实用程序函数：一个用于格式化中间步骤（代理操作、工具输出对）以输入可发送到模型的消息的组件，以及一个用于将输出消息转换为代理操作/代理完成的组件。

In [19]:
from langchain.agents.json_chat.prompt import TEMPLATE_TOOL_RESPONSE

TEMPLATE_TOOL_RESPONSE = """工具响应：
----------------------------------

{observation}

用户的输入：
----------------------------------
请根据工具的响应判断，是否能够回答问题：

{input}

请根据工具响应的内容，思考接下来的回复。回复格式严格按照前面所说的2种JSON回复格式，
选择其中1种进行回复。请记住通过单个选项，以JSON模式格式化回复内容，不要回复其他内容。
"""

In [20]:
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from typing import List


def format_log_to_messages(
    query,
    intermediate_steps,
    template_tool_response
):
    """构建思考过程"""
    thoughts: List[BaseMessage] = []
    for action, observation in intermediate_steps:
        thoughts.append(AIMessage(content=action.log))
        human_message = HumanMessage(
            content=template_tool_response.format(
                input=query,
                observation=observation
            )
        )
        thoughts.append(human_message)
    return thoughts

In [21]:
from langchain.agents.agent import AgentOutputParser
from langchain_core.output_parsers.json import parse_json_markdown
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish
import logging

logger = logging.getLogger(__name__)

class JSONAgentOutputParser(AgentOutputParser):
    """Parses tool invocations and final answers in JSON format.

    Expects output to be in one of two formats.

    If the output signals that an action should be taken,
    should be in the below format. This will result in an AgentAction
    being returned.

    ```
    {
      "action": "search",
      "action_input": "2+2"
    }
    ```

    If the output signals that a final answer should be given,
    should be in the below format. This will result in an AgentFinish
    being returned.

    ```
    {
      "action": "Final Answer",
      "answer": "4"
    }
    ```
    """
    
    def parse(self, text):
        try:
            response = parse_json_markdown(text)
            if isinstance(response, list):
                logger.warning(f"Got mutiple action responses: {response}")
                response = response[0]
            if response.get("action") == "Final Answer":
                return AgentFinish(
                    {"output": response.get("answer")},text
                )
            else:
                return AgentAction(
                    response['action'],
                    response.get("action_input", {}),
                    text
                )
        except OutputParserException as e:
            raise OutputParserException(f"Failed to parse tool call or final answer: {text}") from e
        
    @property
    def _type(self) -> str:
        return "json-agent"
    

In [22]:
from langchain_core.runnables import RunnablePassthrough

agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_messages(
            x["input"], 
            x["intermediate_steps"], 
            template_tool_response = TEMPLATE_TOOL_RESPONSE
        )
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_messages(x['input'], x['intermediate_steps'], template_tool_response=TEMPLATE_TOOL_RESPONSE))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langc

In [23]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [24]:
agent_executor.invoke({"input": "打开浏览器，用谷歌搜索刘亦菲"})



> Entering new AgentExecutor chain...
```json
{
    "reason": "运行shell命令以在计算机上打开网络浏览器并搜索刘亦菲",
    "action": "run_shell",
    "action_input": "open -a 'Google Chrome' 'https://www.google.com/search?q=刘亦菲'"
}
```Executing command:
 ["open -a 'Google Chrome' 'https://www.google.com/search?q=刘亦菲'"]


/Users/libing/miniconda3/envs/kk_langchain/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


```json
{
    "action": "Final Answer",
    "answer": "已打开网络浏览器并搜索了刘亦菲"
}
```

> Finished chain.


{'input': '打开浏览器，用谷歌搜索刘亦菲', 'output': '已打开网络浏览器并搜索了刘亦菲'}

In [26]:
agent_executor.invoke({"input": "打开浏览器，用谷歌搜索黑神话·悟空"})



> Entering new AgentExecutor chain...
```json
{
    "reason": "运行shell命令来在计算机上打开一个网页",
    "action": "run_shell",
    "action_input": "open -a 'Google Chrome' 'https://www.google.com/search?q=黑神话·悟空'"
}
```Executing command:
 ["open -a 'Google Chrome' 'https://www.google.com/search?q=黑神话·悟空'"]


/Users/libing/miniconda3/envs/kk_langchain/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


```json
{
    "action": "Final Answer",
    "answer": "已打开浏览器并用谷歌搜索了黑神话·悟空"
}
```

> Finished chain.


{'input': '打开浏览器，用谷歌搜索黑神话·悟空', 'output': '已打开浏览器并用谷歌搜索了黑神话·悟空'}

## 添加历史记录

这太棒了 - 我们有一个代理！但是，此代理是无状态的 - 它不记得有关先前交互的任何信息。这意味着你不能轻易地提出后续问题。让我们通过添加内存来解决这个问题。

为此，我们需要做两件事：

1. 在提示符中为内存变量添加一个位置
2. 跟踪聊天记录

首先，让我们在提示中添加一个内存位置。为此，我们通过为带有 键 "chat_history" 的消息添加一个占位符 .请注意，我们将其放在新用户输入的上方（以遵循对话流）。

In [27]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是非常强大的助手，你可以使用各种工具来完成人类交给的问题和任务。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", promptTemplate),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag

In [28]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools])
)


agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_messages(
            x["input"], 
            x["intermediate_steps"], 
            template_tool_response = TEMPLATE_TOOL_RESPONSE
        )
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [30]:
chat_history.extend(
    [
        HumanMessage(content="你好，我是老k，一名顶级黑客程序员。"),
        AIMessage(content="你好，老k，很高兴认识你。")
    ]
)

agent_executor.invoke({"input": "我叫什么？什么职业？爱好是什么？", "chat_history": chat_history})



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "answer": "你叫老k。你的职业是一名顶级黑客程序员。你的爱好是编程和破解系统。"
}
```

> Finished chain.


{'input': '我叫什么？什么职业？爱好是什么？',
 'chat_history': [HumanMessage(content='你好，我是老k，一名顶级黑客程序员。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，老k，很高兴认识你。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='你好，我是老k，一名顶级黑客程序员。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，老k，很高兴认识你。', additional_kwargs={}, response_metadata={})],
 'output': '你叫老k。你的职业是一名顶级黑客程序员。你的爱好是编程和破解系统。'}